<a href="https://colab.research.google.com/github/marisbotero/ML/blob/master/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install statsmodels --upgrade

     |████████████████████████████████| 8.7MB 2.8MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [0]:

import statsmodels.api as sm

In [0]:
from statsmodels.tsa.ar_model import AutoReg

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [0]:
import pandas as pd

In [0]:
# load dataset
series = pd.read_excel('serie.xlsx', header=0, index_col=0, parse_dates=True, squeeze=True)

In [114]:
series

Fecha
2020-02-01    1143785
2020-02-02    1144351
2020-02-03    1150719
2020-02-04    1157215
2020-02-05    1161932
               ...   
2020-05-09    1126401
2020-05-10    1126392
2020-05-11    1126442
2020-05-12    1124785
2020-05-13    1123585
Name: total, Length: 103, dtype: int64

In [0]:
X = series.values

In [0]:
train, test = X[1:len(X)-2], X[len(X)-2:]

In [0]:
# train autoregression
model = AutoReg(train, lags=29)
model_fit = model.fit()

In [118]:
import matplotlib.pyplot as plt
%matplotlib inline
print('Coefficients: %s' % model_fit.params)
# make predictions
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)

Coefficients: [ 7.97235016e+04  9.82901865e-01  2.18656817e-03  6.55300129e-02
 -1.97904095e-01  7.29932091e-02  2.56332987e-01 -5.09300851e-02
  2.57121501e-02 -2.38743924e-01 -5.74077527e-02  3.91083628e-02
  1.07403380e-01  1.30780198e-01 -1.81658545e-01 -3.81913002e-03
 -5.11507434e-02  3.92242820e-01 -3.58323474e-01 -2.15359303e-01
  8.03542069e-02  2.25044144e-01 -5.59849454e-03  1.13638029e-01
 -8.63220416e-02 -3.62675486e-02  6.22245669e-02  9.72835561e-02
 -1.89485921e-01 -4.88960621e-02]


In [119]:
for i in range(len(predictions)):
	print('predicted=%f, expected=%f' % (predictions[i], test[i]))
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

predicted=1126804.728081, expected=1124785.000000
predicted=1125513.179729, expected=1123585.000000
Test RMSE: 1974.485


In [120]:
predictions

array([1126804.72808098, 1125513.17972857])

In [0]:

df=pd.DataFrame(predictions, columns=['a'])

In [122]:
df

,a
0,1.126805e+06
1,1.125513e+06


In [0]:
df.to_csv('predict_mayo_5.csv', encoding='utf-8')

In [0]:
X = series.values
train, test = X[1:len(X)-7], X[len(X)-7:]
# train autoregression
window = 29
model = AutoReg(train, lags=29)
model_fit = model.fit()
coef = model_fit.params

In [100]:
history = train[len(train)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
for t in range(len(test)):
	length = len(history)
	lag = [history[i] for i in range(length-window,length)]
	yhat = coef[0]
	for d in range(window):
		yhat += coef[d+1] * lag[window-d-1]
	obs = test[t]
	predictions.append(yhat)
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

predicted=1130718.825813, expected=1127912.000000
predicted=1123813.550261, expected=1125591.000000
predicted=1125025.508637, expected=1126401.000000
predicted=1127338.685251, expected=1126392.000000
predicted=1128451.452410, expected=1126442.000000
predicted=1126638.726420, expected=1124785.000000
predicted=1123898.279319, expected=1123585.000000
Test RMSE: 1748.392


In [0]:
df2=pd.DataFrame(predictions, columns=['a'])

In [102]:
df2.head(10)

,a
0,1.130719e+06
1,1.123814e+06
2,1.125026e+06
3,1.127339e+06
4,1.128451e+06
5,1.126639e+06
6,1.123898e+06


In [103]:
df2.shape

(7, 1)

In [0]:
df2.to_csv('predict_mayo_6.csv', encoding='utf-8')

In [123]:
!pip install pmdarima
from pmdarima.arima import auto_arima

     |████████████████████████████████| 1.5MB 2.7MB/s 


In [0]:
# load dataset
df = pd.read_excel('serie.xlsx')

In [125]:
df.head()

,Fecha,total
0,2020-02-01,1143785
1,2020-02-02,1144351
2,2020-02-03,1150719
3,2020-02-04,1157215
4,2020-02-05,1161932


In [126]:
model_auto = auto_arima(df.total, m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, information_criterion = 'aic')

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [127]:
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(1, 2, 0), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(1, 0, 0, 5),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=True)

In [128]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                                 y   No. Observations:                  103
Model:             SARIMAX(1, 2, 0)x(1, 0, 0, 5)   Log Likelihood                -943.252
Date:                           Fri, 15 May 2020   AIC                           1894.504
Time:                                   02:01:03   BIC                           1904.964
Sample:                                        0   HQIC                          1898.739
                                           - 103                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -84.2071    245.551     -0.343      0.732    -565.478     397.063
ar.L1         -0.0069      0.022     -0.314      0.753      -0.050       0.036
ar.S.L5        0.0038      0.002      2.085      0.037       0.000       0.007
sigma2      6.631e+06      0.026   2.54e+08      0.000    6.63e+06    6.63e+06
===================================================================================
Ljung-Box (Q):                       76.05   Jarque-Bera (JB):                31.05
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.71   Skew:                             0.61
Prob(H) (two-sided):                  0.32   Kurtosis:                         5.43
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 1.19e+24. Standard errors may be unstable.
"""

In [129]:
#Creando el indice del futuro
date = pd.to_datetime("05-14-2020")
date

Timestamp('2020-05-14 00:00:00')

In [0]:
import numpy as np
i_date=date + pd.to_timedelta(np.arange(50), 'D')

In [138]:
i_date

DatetimeIndex(['2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17',
               '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21',
               '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25',
               '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29',
               '2020-05-30', '2020-05-31', '2020-06-01', '2020-06-02',
               '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-06',
               '2020-06-07', '2020-06-08', '2020-06-09', '2020-06-10',
               '2020-06-11', '2020-06-12', '2020-06-13', '2020-06-14',
               '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18',
               '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22',
               '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26',
               '2020-06-27', '2020-06-28', '2020-06-29', '2020-06-30',
               '2020-07-01', '2020-07-02'],
              dtype='datetime64[ns]', freq=None)

In [0]:
predictions=model_auto.predict(len(i_date))

In [140]:
df_test = pd.Series(predictions, index=i_date)
df_test.head()

2020-05-14    1.122310e+06
2020-05-15    1.120947e+06
2020-05-16    1.119502e+06
2020-05-17    1.117966e+06
2020-05-18    1.116349e+06
dtype: float64

In [0]:
df_auto_pred = pd.DataFrame({'Predictions': predictions},
                            index = i_date)

In [143]:
df_auto_pred.head(10)

,Predictions
2020-05-14,1.122310e+06
2020-05-15,1.120947e+06
2020-05-16,1.119502e+06
2020-05-17,1.117966e+06
2020-05-18,1.116349e+06
2020-05-19,1.114647e+06
2020-05-20,1.112862e+06
2020-05-21,1.110992e+06
2020-05-22,1.109039e+06
2020-05-23,1.107002e+06


In [0]:

df_auto_pred.to_csv('predict_mayo_7.csv', encoding='utf-8')